## Linear Regression

# imports
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split



In [1]:
# Read the Dataframe here


In [ ]:
# Target selection
X = dataframe.drop('arr_delay',axis=1)
y = dataframe['arr_delay']

In [ ]:
# Standard Scaler creation using the fit_transform() method
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)

In [ ]:
# Splite DataFrame to Train and Test sets
X_test, X_train, y_test, y_train = train_test_split(scaled_df, y, test_size=0.30)

In [ ]:
# Fitting the model
LR_model = linear_model.LinearRegression(learning_rate=0.001 )
LR_model.fit(X_train,y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# The coefficients
print('Coefficients: \n', LR_model.coef_)

# The mean squared error
print('Mean squared error: %.2f \n'
      % mean_squared_error(y_test, y_pred))

# The coefficient of determination: 
print('Coefficient of determination: %.2f \n'
      % r2_score(y_test, y_pred))

### Implementing Ridge

In [ ]:
# Ridge regression
Rid_regression = Ridge()

In [ ]:
# fit the data
Rid_regression.fit(X_train, y_train)

In [ ]:
ridg_y_pred = Rid_regression.predict(X_test)

In [ ]:
# The coefficients
print('Coefficients: \n', Rid_regression.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, ridg_y_pred))


# The coefficient of determination: 
print('Coefficient of determination: %.2f'
      % r2_score(y_test, ridg_y_pred))